In [3]:
from cluster_scripts import *
from jup_plot import *

In [4]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D



def radar_factory(num_vars, frame='circle',rotation=0):
    """
    Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle', 'polygon'}
        Shape of frame surrounding axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False) 

    class RadarTransform(PolarAxes.PolarTransform):

        def transform_path_non_affine(self, path):
            # Paths with non-unit interpolation steps correspond to gridlines,
            # in which case we force interpolation (to defeat PolarTransform's
            # autoconversion to circular arcs).
            if path._interpolation_steps > 1:
                path = path.interpolated(num_vars)
            return Path(self.transform(path.vertices), path.codes)

    class RadarAxes(PolarAxes):

        name = 'radar'
        PolarTransform = RadarTransform

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')
            self.set_theta_offset(rotation)


        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.append(x, x[0])
                y = np.append(y, y[0])
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k",orientation=rotation)
                # return RegularPolygon((0.5, 0.5), num_vars,
                #                       radius=.5, edgecolor="k")
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                # spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                #                     + self.transAxes)
                spine.set_transform(Affine2D().rotate(rotation).scale(.5).translate(.5, .5)
                                    + self.transAxes)
                # print(spine)
                return {'polar': spine}
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

In [5]:
xc_functional_list = ['PBE','PBE-D2','PBE-D2-Ne','PBE-DDSC','PBE-D3','PBE-TS','PBE-TSHI','PBE-D4','PBE-MBDHI','PBE-MBDFI','revPBE','revPBE-D3','revPBE-D4','PBEsol','PBEsol-D3','PBEsol-D4','vdW-DF','vdW-DF2','optB86b-vdW','rev-vdW-DF2','R2SCAN','R2SCAN-D3','R2SCAN-D4','SCAN-rVV10','PBE0','PBE0-D3','PBE0-D4','PBE0-TSHI','HSE06','HSE06-D4','B3LYP','B3LYP-D2','B3LYP-D2-Ne','B3LYP-D3','B3LYP-D4']

full_methods_list = xc_functional_list + ['B2PLYP','HF','MP2','CCSD','CCSD(T)']

xc_rel_ene_list = { x: { y: {z : [] for z in full_methods_list} for y in [4,6,8]}  for x in ['Au','Ag','Cu']}
xc_bind_ene_list = { x: { y: {z : [] for z in full_methods_list} for y in [4,6,8]}  for x in ['Au','Ag','Cu']}

xc_rel_ene_mad = { x: { y: {z : [] for z in full_methods_list} for y in [4,6,8,'Total']}  for x in ['Au','Ag','Cu','Total']}
xc_bind_ene_mad = { x: { y: {z : [] for z in full_methods_list} for y in [4,6,8]}  for x in ['Au','Ag','Cu']}


for i in ['Au','Ag','Cu']:
    for j in [4,6,8]:
        for k in xc_functional_list:
            if k == 'optB86b-vdW':
                func_name = 'optB86b-vdw'
            elif k == 'rev-vdW-DF2':
                func_name = 'rev-vdw-DF2'
            elif k == 'PBE-D2-Ne':
                func_name = 'PBE-D2'.lower()
            elif k == 'B3LYP-D2-Ne':
                func_name = 'B3LYP-D2'.lower()
    
            else:
                func_name = k.lower()
            
            ene_0 = find_energy('Data/01-Gas_Cluster/DFT/{0}{1}/1/{2}/AD/OUTCAR'.format(i,j,func_name),code_format='vasp')
            for l in [1,2,3,4]:
                ene_ad= find_energy('Data/01-Gas_Cluster/DFT/{0}{1}/{3}/{2}/AD/OUTCAR'.format(i,j,func_name,l),code_format='vasp')

                xc_rel_ene_list[i][j][k] += [(ene_ad - ene_0)*1000]
                xc_bind_ene_list[i][j][k] += [(ene_ad)*1000]
            
        for l in [1,2,3,4]:
        
            # if (j == 3 and i == 'Ag') or (j == 3 and i == 'Cu'):
            #     continue
            
            ene_hf = []
            ene_mp2 = []
            ene_ccsd = []
            ene_ccsdt = []
            ene_dft = []
            ene_b2plyp = [] 
            for m in [1,3,2]:
                root_folder = 'Data/01-Gas_Cluster/cWFT/LCCSDT_From_PBE/{0}{1}/{2}/{3}/mrcc.out'.format(i,j,l,m)
                root_folder_b2plyp = 'Data/01-Gas_Cluster/cWFT/B2PLYP/{0}{1}/{2}/{3}/mrcc.out'.format(i,j,l,m)

                ene_hf += [find_energy(root_folder,code_format='mrcc',typ='hf')]
                ene_mp2 += [find_energy(root_folder,code_format='mrcc',typ='lmp2')]
                ene_ccsd += [find_energy(root_folder,code_format='mrcc',typ='lccsd')]
                ene_ccsdt += [find_energy(root_folder,code_format='mrcc',typ='lccsdt')]
                ene_dft += [find_energy(root_folder_b2plyp,code_format='mrcc',typ='hf')]
                ene_b2plyp += [find_energy(root_folder_b2plyp,code_format='mrcc',typ='B2PLYP')]


            dummy_dict ={
                'MP2': extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=3,Y=4,family='acc',\
                convert_Hartree=False ,shift=0.0,output=False),
                'CCSD': extrapolate.get_cbs(ene_hf[0],ene_ccsd[0],ene_hf[1],ene_ccsd[1],X=3,Y=4,family='acc',\
                convert_Hartree=False ,shift=0.0,output=False),
                'CCSD(T)': extrapolate.get_cbs(ene_hf[0],ene_ccsdt[0],ene_hf[1],ene_ccsdt[1],X=3,Y=4,family='acc',\
                convert_Hartree=False ,shift=0.0,output=False),
                'B2PLYP': extrapolate.get_cbs(ene_dft[0],ene_b2plyp[0],ene_dft[1],ene_b2plyp[1],X=3,Y=4,family='acc',\
                convert_Hartree=False ,shift=0.0,output=False)
            }
            # dummy_mp2 = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=3,Y=4,family='acc',\
            #     convert_Hartree=False ,shift=0.0,output=False)
            # dummy_ccsd = extrapolate.get_cbs(ene_hf[0],ene_ccsd[0],ene_hf[1],ene_ccsd[1],X=3,Y=4,family='acc',\
            #     convert_Hartree=False ,shift=0.0,output=False)
            # dummy_ccsdt = extrapolate.get_cbs(ene_hf[0],ene_ccsdt[0],ene_hf[1],ene_ccsdt[1],X=3,Y=4,family='acc',\
            #     convert_Hartree=False ,shift=0.0,output=False)
            # dummy_b2plyp = extrapolate.get_cbs(ene_dft[0],ene_b2plyp[0],ene_dft[1],ene_b2plyp[1],X=3,Y=4,family='acc',\
            #     convert_Hartree=False ,shift=0.0,output=False)



            xc_bind_ene_list[i][j]['HF'] += [dummy_dict['MP2'][0]]
            xc_bind_ene_list[i][j]['MP2'] += [dummy_dict['MP2'][-1] + ene_mp2[-1] - ene_mp2[0]]
            xc_bind_ene_list[i][j]['CCSD'] += [dummy_dict['CCSD'][-1] + ene_ccsd[-1] - ene_ccsd[0]]
            xc_bind_ene_list[i][j]['CCSD(T)'] += [dummy_dict['CCSD(T)'][-1] + ene_ccsdt[-1] - ene_ccsdt[0]]
            xc_bind_ene_list[i][j]['B2PLYP'] += [dummy_dict['B2PLYP'][-1] + ene_b2plyp[-1] - ene_b2plyp[0]]

            # xc_rel_ene_list[i][j]['HF'] += [(dummy_dict['MP2'][0] - xc_bind_ene_list[i][j]['HF'][0])*1000]

            for m in ['HF','MP2','CCSD','CCSD(T)','B2PLYP']:
                xc_rel_ene_list[i][j][m] += [(xc_bind_ene_list[i][j][m][-1]- xc_bind_ene_list[i][j][m][0])*1000*Hartree]



In [6]:
for j in xc_functional_list + ['B2PLYP','HF','MP2','CCSD']:
    for i in ['Au','Ag','Cu']:
        for k in [4,6,8]:
            xc_rel_ene_mad[i][k][j] = np.sum([abs(xc_rel_ene_list[i][k][j][l] - xc_rel_ene_list[i][k]['CCSD(T)'][l]) for l in list(range(1,4))])/3
        xc_rel_ene_mad[i]['Total'][j] = np.sum([xc_rel_ene_mad[i][l][j] for l in [4,6,8]])/3
        xc_rel_ene_mad['Total'][4][j] = np.sum([xc_rel_ene_mad[l][4][j] for l in ['Au','Ag','Cu']])/3
        xc_rel_ene_mad['Total'][6][j] = np.sum([xc_rel_ene_mad[l][6][j] for l in ['Au','Ag','Cu']])/3
        xc_rel_ene_mad['Total'][8][j] = np.sum([xc_rel_ene_mad[l][8][j] for l in ['Au','Ag','Cu']])/3

    # print(np.sum([xc_rel_ene_mad[l]['Total'][j] for l in ['Au','Ag','Cu']])/3,np.sum([xc_rel_ene_mad['Total'][l][j] for l in [4,6,8]])/3)

    xc_rel_ene_mad['Total']['Total'][j] = np.sum([xc_rel_ene_mad[l]['Total'][j] for l in ['Au','Ag','Cu']])/3
    # m2_mgo_xc_bind_ene_mad['Total'][j] = (m2_mgo_xc_bind_ene_mad['Au'][j] + m2_mgo_xc_bind_ene_mad['Ag'][j] + m2_mgo_xc_bind_ene_mad['Cu'][j])/3

/home/shixubenjamin/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [7]:
xc_rel_ene_mad['Total']['Total']

{'PBE': 199.4339831527975,
 'PBE-D2': 152.606379898664,
 'PBE-D2-Ne': 152.606379898664,
 'PBE-DDSC': 142.89887646256486,
 'PBE-D3': 129.56216720330548,
 'PBE-TS': 189.92743278242668,
 'PBE-TSHI': 190.67397646570657,
 'PBE-D4': 120.14120310880718,
 'PBE-MBDHI': 123.35296088658544,
 'PBE-MBDFI': 111.02066718288167,
 'revPBE': 312.7237027179685,
 'revPBE-D3': 155.31159044172244,
 'revPBE-D4': 146.25141495545134,
 'PBEsol': 89.44712423313649,
 'PBEsol-D3': 89.98219787506962,
 'PBEsol-D4': 88.86050083803217,
 'vdW-DF': 359.24416494019096,
 'vdW-DF2': 451.9232098194639,
 'optB86b-vdW': 103.35732006366578,
 'rev-vdW-DF2': 138.53399311947877,
 'R2SCAN': 126.39968425039848,
 'R2SCAN-D3': 119.14804054669865,
 'R2SCAN-D4': 121.22378536151741,
 'SCAN-rVV10': 137.73897047260635,
 'PBE0': 190.50274081733914,
 'PBE0-D3': 117.57896711363526,
 'PBE0-D4': 122.46989674326504,
 'PBE0-TSHI': 182.75151155807944,
 'HSE06': 190.60894944909367,
 'HSE06-D4': 123.80139574538977,
 'B3LYP': 406.0457087083532,
 'B3

In [8]:
fig, axs = plt.subplots(1,3,figsize=(6.69,6),dpi=600, sharey=True,sharex=True,constrained_layout=True)


eov_method_sorted_list = xc_functional_list + ['B2PLYP','HF','MP2','CCSD']

# axs.
eov_method_sorted_list_val = np.array(list(range(len(eov_method_sorted_list))))

width=0.35

counter = 0
for i in ['Au','Ag','Cu']:


    axs[counter].barh(np.array(list(range(len(eov_method_sorted_list))))*1.5+width, [ (xc_rel_ene_mad[i][4][x]) for x in eov_method_sorted_list],width,label=r'Tetramer',color=color_dict['red'])
    axs[counter].barh(np.array(list(range(len(eov_method_sorted_list))))*1.5, [ (xc_rel_ene_mad[i][6][x]) for x in eov_method_sorted_list],width,label=r'Hexamer',color=color_dict['blue'])
    axs[counter].barh(np.array(list(range(len(eov_method_sorted_list))))*1.5 -width, [ (xc_rel_ene_mad[i][8][x]) for x in eov_method_sorted_list],width,label=r'Octamer',color=color_dict['green'])

    axs[counter].plot([ (xc_rel_ene_mad[i]['Total'][x]) for x in eov_method_sorted_list],np.array(list(range(len(eov_method_sorted_list))))*1.5, marker='s',markeredgewidth=1.0, markerfacecolor="None",color=color_dict['black'], linewidth=1,label='Total')

    axs[counter].set_xlabel('MAD (meV)')

    axs[counter].set_title('{0} Gas Clusters'.format(i))

    counter += 1

# axs[1].barh(np.array(list(range(len(eov_method_sorted_list))))*1.5-width, [ (m2_bind_ene_mad['Au'][x]) for x in eov_method_sorted_list],width,label=r'Au',color=color_dict['orange'])
# axs[1].barh(np.array(list(range(len(eov_method_sorted_list))))*1.5, [ (m2_bind_ene_mad['Ag'][x]) for x in eov_method_sorted_list],width,label=r'Ag',color=color_dict['grey'])
# axs[1].barh(np.array(list(range(len(eov_method_sorted_list))))*1.5 +width, [ (m2_bind_ene_mad['Cu'][x]) for x in eov_method_sorted_list],width,label=r'Cu',color=color_dict['brown'])

# axs[1].plot(m2_tot_bind_ene_mad,np.array(list(range(len(eov_method_sorted_list))))*1.5, marker='s',markeredgewidth=1.0, markerfacecolor="None",color=color_dict['black'], linewidth=1,label='Total')

# axs[0].barh(eov_method_sorted_list, [ (m4_rel_ene_mad['Ag'][x]) for x in eov_method_sorted_list],0.35,label=r'Ag')
# axs[0].barh(eov_method_sorted_list, [ (m4_rel_ene_mad['Cu'][x]) for x in eov_method_sorted_list],0.35,label=r'Cu')

# # axs[0].barh(eov_method_sorted_list, m4_tot_rel_ene_mad, 0.35,label=r'Error in $E_\textrm{int}^\textrm{M-M}$')
# axs[0].plot([0.02,0.04,0.06],[0,1,2])

# axs.bar(eov_method_sorted_list, m2_tot_bind_ene_mad, 0.35, bottom=m4_tot_rel_ene_mad,label=r'Error in $E_\textrm{int}^\textrm{M-S}$')
axs[1].legend()


axs[0].set_yticks(np.array(list(range(len(eov_method_sorted_list))))*1.5)
axs[0].set_yticklabels(eov_method_sorted_list)
axs[0].set_xlim([0,1000])

# plt.xticks(rotation=90)

plt.savefig('Gas_MAD_Size.png')

# plt.savefig('Combined_graph.png')


In [9]:
fig, axs = plt.subplots(figsize=(6.69,3),dpi=600, sharey=True,sharex=True,constrained_layout=True)


eov_method_sorted_list = xc_functional_list + ['B2PLYP','HF','MP2','CCSD']

# axs.
eov_method_sorted_list_val = np.array(list(range(len(eov_method_sorted_list))))

width=0.35

counter = 0
axs.plot(np.array(list(range(len(eov_method_sorted_list)))), [ (xc_rel_ene_mad['Au']['Total'][x]) for x in eov_method_sorted_list], marker='o',markeredgewidth=1.0, markerfacecolor="None",color=color_dict['yellow'], linewidth=1,label='Au',alpha=0.5)
axs.plot(np.array(list(range(len(eov_method_sorted_list)))), [ (xc_rel_ene_mad['Ag']['Total'][x]) for x in eov_method_sorted_list], marker='x',markeredgewidth=1.0, markerfacecolor="None",color=color_dict['grey'], linewidth=1,label='Ag',alpha=0.5)
axs.plot(np.array(list(range(len(eov_method_sorted_list)))), [ (xc_rel_ene_mad['Cu']['Total'][x]) for x in eov_method_sorted_list], marker='^',markeredgewidth=1.0, markerfacecolor="None",color=color_dict['brown'], linewidth=1,label='Cu',alpha=0.5)
axs.plot(np.array(list(range(len(eov_method_sorted_list)))), [ (xc_rel_ene_mad['Total']['Total'][x]) for x in eov_method_sorted_list], marker='s',markeredgewidth=1.0, markerfacecolor="None",color=color_dict['black'], linewidth=1,label='Total')

axs.legend()


axs.set_xticks(np.array(list(range(len(eov_method_sorted_list)))))
axs.set_xticklabels(eov_method_sorted_list,rotation=90)
axs.set_ylim([0,400])

axs.set_ylabel('MAD (meV)')

# plt.xticks(rotation=90)

plt.savefig('Gas_MAD_Overall.png')

# plt.savefig('Combined_graph.png')


In [10]:
eov_method_sorted_list_val = np.array(list(range(len(full_methods_list))))

for i in ['Au','Ag','Cu']:
    for j in [4,6,8]:
        erel_list = {y: [] for y in [1,2,3,4]}
        for k in full_methods_list:
            # print(k)
            counter = 0
            for k1 in [1,2,3,4]:
                erel_list[k1] += [xc_rel_ene_list[i][j][k][counter]]
                counter += 1



      

        fig, axs = plt.subplots(figsize=(6.69,5),dpi=300, constrained_layout=True)

        axs.plot(eov_method_sorted_list_val, erel_list[1] , marker='o',markeredgewidth=1.0, markerfacecolor="None",color=color_dict['black'], linewidth=1,label='Structure 1')


        axs.plot(eov_method_sorted_list_val, erel_list[2] , marker='o',markeredgewidth=1.0, markerfacecolor="None",color=color_dict['red'], linewidth=1,label='Structure 2')

        axs.plot([-1,42], [erel_list[2][-1]]*2 , '--',color=color_dict['red'], linewidth=1.3,alpha=0.7)


        axs.plot(eov_method_sorted_list_val, erel_list[3] , marker='o',markeredgewidth=1.0, markerfacecolor="None",color=color_dict['blue'], linewidth=1,label='Structure 3')

        axs.plot([-1,42], [erel_list[3][-1]]*2 , '--',color=color_dict['blue'], linewidth=1.3,alpha=0.7)

        axs.plot(eov_method_sorted_list_val, erel_list[4] , marker='o',markeredgewidth=1.0, markerfacecolor="None",color=color_dict['green'], linewidth=1,label='Structure 4')

        axs.plot([-1,42], [erel_list[4][-1]]*2 , '--',color=color_dict['green'], linewidth=1.3,alpha=0.7)


        axs.legend()

        axs.set_ylabel('Energy relative to Structure 1 (meV)')
        axs.set_title('{0}{1}'.format(i,j))

        axs.set_xticks(eov_method_sorted_list_val)
        axs.set_xticklabels(full_methods_list,rotation=90)

        axs.set_xlim([-1,40])
        # axs.set_ylim([-0.001,0.005])


        plt.savefig('Gas_Rel_Ene_{0}{1}.png'.format(i,j))
        plt.close()




In [11]:
xc_functional_list = ['PBE','PBE-D2','PBE-D2-Ne','PBE-DDSC','PBE-D3','PBE-TS','PBE-TSHI','PBE-D4','PBE-MBDHI','PBE-MBDFI','revPBE','revPBE-D3','revPBE-D4','PBEsol','PBEsol-D3','PBEsol-D4','vdW-DF','vdW-DF2','optB86b-vdW','rev-vdW-DF2','R2SCAN','R2SCAN-D3','R2SCAN-D4','SCAN-rVV10','PBE0','PBE0-D3','PBE0-D4','PBE0-TSHI','HSE06','HSE06-D4','B3LYP','B3LYP-D2','B3LYP-D2-Ne','B3LYP-D3','B3LYP-D4']

full_methods_list = xc_functional_list + ['B2PLYP','HF','MP2','CCSD','CCSD(T)']

m2_mgo_xc_bind_ene_list = { x: { y: {z : 0 for z in full_methods_list} for y in list(range(1,9))}  for x in ['Au','Ag','Cu']}
m2_mgo_xc_bind_ene_mad = { x: { y: {z : 0 for z in full_methods_list} for y in list(range(1,9)) + ['Total']}  for x in ['Au','Ag','Cu','Total']}

# m2_mgo_xc_bind_ene_mad = { x:  {z : 0 for z in full_methods_list} for x in ['Au','Ag','Cu','Total']}
# m2_mgo_xc_bind_ene_mad = { x:  {z : 0 for z in full_methods_list} for x in ['Au','Ag','Cu','Total']}


for i in ['Au','Ag','Cu']:
    for j in range(1,9): #,8]:
        for k in xc_functional_list:
            if k == 'optB86b-vdW':
                func_name = 'optB86b-vdw'
            elif k == 'rev-vdW-DF2':
                func_name = 'rev-vdw-DF2'
            # elif k == 'PBE-D2-Ne':
            #     func_name = 'PBE-D2'.lower()
            elif k == 'B3LYP-D2-Ne':
                func_name = 'B3LYP-D2'.lower()
    
            else:
                func_name = k.lower()
            
            a= find_energy('Data/02-M2_MgO/DFT/{0}2_MgO/{1}/{2}/AD_SLAB/OUTCAR'.format(i,j,func_name),code_format='vasp')
            b= find_energy('Data/02-M2_MgO/DFT/Au2_MgO/1/{0}/SLAB/OUTCAR'.format(func_name),code_format='vasp')
            c= find_energy('Data/02-M2_MgO/DFT/{0}2_MgO/{1}/{2}/AD/OUTCAR'.format(i,j,func_name),code_format='vasp')
            ene_dummy = (a-b-c)*1000
            # print(i,j,k,ene_dummy)
            m2_mgo_xc_bind_ene_list[i][j][k] = (ene_dummy)
            
    for l in range(1,9):
    
        # if (j == 3 and i == 'Ag') or (j == 3 and i == 'Cu'):
        #     continue
        
        ene_hf = []
        ene_mp2 = []
        ene_ccsd = []
        ene_ccsdt = []
        ene_dft = []
        ene_b2plyp = [] 
        for m in [1,3,2]:
            root_folder = 'Data/02-M2_MgO/cWFT/CC/{0}2_MgO/{1}/{2}'.format(i,l,m)
            # root_folder = 'Data/01-Gas_Cluster/cWFT/LCCSDT_From_PBE/{0}{1}/{2}/{3}/mrcc.out'.format(i,j,l,m)
            root_folder_b2plyp = 'Data/02-M2_MgO/cWFT/B2PLYP/{0}/B2PLYP_{1}/{2}'.format(i,m,j)

            ene_hf += [get_eads(root_folder,code_format='mrcc',typ='hf')]
            ene_mp2 += [get_eads(root_folder,code_format='mrcc',typ='lmp2')]
            ene_ccsd += [get_eads(root_folder,code_format='mrcc',typ='lccsd')]
            ene_ccsdt += [get_eads(root_folder,code_format='mrcc',typ='lccsdt')]
            ene_dft += [get_eads(root_folder_b2plyp,code_format='mrcc',typ='hf')]
            ene_b2plyp += [get_eads(root_folder_b2plyp,code_format='mrcc',typ='B2PLYP')]


        dummy_dict ={
            'MP2': extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=3,Y=4,family='mixcc',\
            convert_Hartree=False ,shift=0.0,output=False),
            'CCSD': extrapolate.get_cbs(ene_hf[0],ene_ccsd[0],ene_hf[1],ene_ccsd[1],X=3,Y=4,family='mixcc',\
            convert_Hartree=False ,shift=0.0,output=False),
            'CCSD(T)': extrapolate.get_cbs(ene_hf[0],ene_ccsdt[0],ene_hf[1],ene_ccsdt[1],X=3,Y=4,family='mixcc',\
            convert_Hartree=False ,shift=0.0,output=False),
            'B2PLYP': extrapolate.get_cbs(ene_dft[0],ene_b2plyp[0],ene_dft[1],ene_b2plyp[1],X=3,Y=4,family='mixcc',\
            convert_Hartree=False ,shift=0.0,output=False)
        }
        # ,
        #     'B2PLYP': extrapolate.get_cbs(ene_dft[0],ene_b2plyp[0],ene_dft[1],ene_b2plyp[1],X=3,Y=4,family='mixcc',\
        #     convert_Hartree=False ,shift=0.0,output=False)
        # }
        # dummy_mp2 = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=3,Y=4,family='acc',\
        #     convert_Hartree=False ,shift=0.0,output=False)
        # dummy_ccsd = extrapolate.get_cbs(ene_hf[0],ene_ccsd[0],ene_hf[1],ene_ccsd[1],X=3,Y=4,family='acc',\
        #     convert_Hartree=False ,shift=0.0,output=False)
        # dummy_ccsdt = extrapolate.get_cbs(ene_hf[0],ene_ccsdt[0],ene_hf[1],ene_ccsdt[1],X=3,Y=4,family='acc',\
        #     convert_Hartree=False ,shift=0.0,output=False)
        # dummy_b2plyp = extrapolate.get_cbs(ene_dft[0],ene_b2plyp[0],ene_dft[1],ene_b2plyp[1],X=3,Y=4,family='acc',\
        #     convert_Hartree=False ,shift=0.0,output=False)



        m2_mgo_xc_bind_ene_list[i][l]['HF'] = (dummy_dict['MP2'][0])*1000*Hartree
        m2_mgo_xc_bind_ene_list[i][l]['MP2'] = (dummy_dict['MP2'][-1] + ene_mp2[-1] - ene_mp2[0])*1000*Hartree
        m2_mgo_xc_bind_ene_list[i][l]['CCSD'] = (dummy_dict['CCSD'][-1] + ene_ccsd[-1] - ene_ccsd[0])*1000*Hartree
        m2_mgo_xc_bind_ene_list[i][l]['CCSD(T)'] =  (dummy_dict['CCSD(T)'][-1] + ene_ccsdt[-1] - ene_ccsdt[0])*1000*Hartree
        m2_mgo_xc_bind_ene_list[i][l]['B2PLYP'] = (dummy_dict['B2PLYP'][-1] + ene_b2plyp[-1] - ene_b2plyp[0])*1000*Hartree

        # xc_rel_ene_list[i][j]['HF'] += [(dummy_dict['MP2'][0] - xc_bind_ene_list[i][j]['HF'][0])*1000]

        # for m in ['HF','MP2','CCSD','CCSD(T)','B2PLYP']:
        #     xc_rel_ene_list[i][j][m] += [(xc_bind_ene_list[i][j][m][-1]- xc_bind_ene_list[i][j][m][0])*1000*Hartree]



    

# for j in full_methods_list:




In [12]:
for j in xc_functional_list + ['B2PLYP','HF','MP2','CCSD']:
    for i in ['Au','Ag','Cu']:
        for k in range(1,9):
            m2_mgo_xc_bind_ene_mad[i][k][j] = abs(m2_mgo_xc_bind_ene_list[i][k][j] - m2_mgo_xc_bind_ene_list[i][k]['CCSD(T)'])
        m2_mgo_xc_bind_ene_mad[i]['Total'][j] = np.sum([abs(m2_mgo_xc_bind_ene_list[i][k][j] - m2_mgo_xc_bind_ene_list[i][k]['CCSD(T)']) for k in list(range(1,9))])/8

    m2_mgo_xc_bind_ene_mad['Total']['Total'][j] = (m2_mgo_xc_bind_ene_mad['Au']['Total'][j] + m2_mgo_xc_bind_ene_mad['Ag']['Total'][j] + m2_mgo_xc_bind_ene_mad['Cu']['Total'][j])/3
    for k in range(1,9):
        m2_mgo_xc_bind_ene_mad['Total'][k][j] = np.sum([abs(m2_mgo_xc_bind_ene_list[i][k][j] - m2_mgo_xc_bind_ene_list[i][k]['CCSD(T)']) for i in ['Au','Ag','Cu']])/3

In [13]:
for i in range(1,9):
    # print(m2_mgo_xc_bind_ene_list['Au'][i])
    print(m2_mgo_xc_bind_ene_list['Au'][i]['CCSD(T)'],m2_mgo_xc_bind_ene_list['Au'][i]['R2SCAN'],m2_mgo_xc_bind_ene_list['Au'][i]['PBE']) #,m2_mgo_xc_bind_ene_list['Au'][i]['PBE-D2-Ne'])

-1695.3544018451694 -1739.464440000006 -1427.7001399999572
-229.1836493303647 -418.65793999997436 -304.5358299999576
-870.8084300067896 -999.7603999999853 -779.2797299999106
-431.27441053979953 -628.0762999999752 -368.57562000000723
-355.0307078975272 -548.6260099998646 -299.8099800000582
-879.5039679433698 -1050.2170599999658 -700.2423100000694
-587.8701305361066 -808.9518799998814 -496.2367099999758
-587.2207334422704 -733.1843400000508 -459.36902000002755


In [14]:
m2_mgo_xc_bind_ene_mad['Total']

{1: {'PBE': 194.65268487675974,
  'PBE-D2': 335.2921784567554,
  'PBE-D2-Ne': 96.6326051231725,
  'PBE-DDSC': 87.25465178997392,
  'PBE-D3': 248.43731178998155,
  'PBE-TS': 280.877911789908,
  'PBE-TSHI': 63.27987302002733,
  'PBE-D4': 132.1037351233109,
  'PBE-MBDHI': 63.38997635337913,
  'PBE-MBDFI': 194.10716178994608,
  'revPBE': 483.7988548768615,
  'revPBE-D3': 349.14587512333543,
  'revPBE-D4': 71.78196845657048,
  'PBEsol': 123.93707178987025,
  'PBEsol-D3': 561.8180917898746,
  'PBEsol-D4': 337.32841512337444,
  'vdW-DF': 289.00818154338464,
  'vdW-DF2': 307.8557648767675,
  'optB86b-vdW': 138.85204178992686,
  'rev-vdW-DF2': 30.95070845656456,
  'R2SCAN': 101.94451512324547,
  'R2SCAN-D3': 335.1491117898638,
  'R2SCAN-D4': 242.96625845666836,
  'SCAN-rVV10': 369.8011951233297,
  'PBE0': 320.444104876548,
  'PBE0-D3': 162.21358178985398,
  'PBE0-D4': 24.475726353397665,
  'PBE0-TSHI': 83.80407487687717,
  'HSE06': 327.7339515433311,
  'HSE06-D4': 23.312964557257448,
  'B3LYP':

In [15]:
xc_rel_ene_mad

{'Au': {4: {'PBE': 210.77479981309673,
   'PBE-D2': 236.76334314642963,
   'PBE-D2-Ne': 236.76334314642963,
   'PBE-DDSC': 184.77159981309651,
   'PBE-D3': 187.18657647976283,
   'PBE-TS': 213.11044981309593,
   'PBE-TSHI': 211.51018981309593,
   'PBE-D4': 182.84894647976262,
   'PBE-MBDHI': 191.784399813096,
   'PBE-MBDFI': 184.0593198130964,
   'revPBE': 293.9444098130961,
   'revPBE-D3': 245.7092831464296,
   'revPBE-D4': 241.46594981309588,
   'PBEsol': 93.66550124703774,
   'PBEsol-D3': 82.0857412470377,
   'PBEsol-D4': 86.34293458037102,
   'vdW-DF': 338.62512314642976,
   'vdW-DF2': 381.3207731464292,
   'optB86b-vdW': 160.00463981309653,
   'rev-vdW-DF2': 175.9087731464293,
   'R2SCAN': 166.87326981307956,
   'R2SCAN-D3': 162.74261314642845,
   'R2SCAN-D4': 163.8296764797677,
   'SCAN-rVV10': 156.8310879137106,
   'PBE0': 175.99197314642927,
   'PBE0-D3': 152.70103981309722,
   'PBE0-D4': 153.5816864797631,
   'PBE0-TSHI': 175.89658981309537,
   'HSE06': 179.31449647976282,
   

In [16]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [81]:
# Now plotting the graphs
from jup_plot import *
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib

# plt.rcparams.update('fontsize': 12)

mm_to_inch=0.0393701

fig, axs = plt.subplots(figsize=(130*mm_to_inch,55*mm_to_inch),dpi=1200, constrained_layout=True)

xc_functional_list = ['PBE','SCAN-rVV10','PBE0-TSHI','B3LYP']

gga_xc_list = ['PBE','PBE-D2','PBE-D3','PBE-D4','PBE-MBDFI'] #['PBE','PBE-D2','PBE-D2-Ne','PBE-DDSC','PBE-D3','PBE-TS','PBE-TSHI','PBE-D4','PBE-MBDHI','PBE-MBDFI']
vdw_inc_xc_list = [] #['vdW-DF','rev-vdW-DF2','optB86b-vdW']
metagga_xc_list = ['R2SCAN','R2SCAN-D3','R2SCAN-D4','SCAN-rVV10']
hybrid_xc_list = ['PBE0','PBE0-D3','PBE0-D4','PBE0-TSHI','B3LYP']
cwft_xc_list = ['B2PLYP','MP2','CCSD']

eov_method_sorted_list = gga_xc_list + vdw_inc_xc_list + metagga_xc_list + hybrid_xc_list + cwft_xc_list

#xc_functional_list + ['B2PLYP','HF','MP2','CCSD']

eov_method_sorted_list_val = np.concatenate((np.arange(len(gga_xc_list)),10 + np.arange(len(vdw_inc_xc_list)), 6 + np.arange(len(metagga_xc_list)), 11 + np.arange(len(hybrid_xc_list)), 17 + np.arange(len(cwft_xc_list))))
                                              


# np.append(list(range(len(gga_xc_list))),list(range(len(gga_xc_list), len(vdw_inc_xc_list))) + list(range(len(vdw_inc_xc_list), len()))

# eov_method_sorted_list_val = np.array(list(range(len(eov_method_sorted_list))))


width = 0.5


axs.bar(eov_method_sorted_list_val*2, [m2_mgo_xc_bind_ene_mad['Total']['Total'][x] for x in eov_method_sorted_list],1.5, linewidth=1, alpha=0.7, color=color_dict['yellow'])
axs.bar(eov_method_sorted_list_val*2, [m2_mgo_xc_bind_ene_mad['Total']['Total'][x] for x in eov_method_sorted_list],1.5, linewidth=1.5, color='none',edgecolor='k')

axs.set_xticks(eov_method_sorted_list_val*2)

# axs.set_title('Metal-Substrate')
axs.set_ylabel('MAD w.r.t. SKZCAM [meV]')



axs.set_xticklabels(eov_method_sorted_list,rotation=50,ha="right")

axs.set_ylim([0,600])

axs.set_xlim([-2,40])



# axs.set_ylim([0,0.6])


plt.savefig('ICQC.png')


In [24]:
print(eov_method_sorted_list_val)

[ 0  1  2  3  4  5  6  7  8  9 11 12 13 15 16 17 18 20 21 22 23 24 25 26
 27 29 30 31]


In [41]:
for x in eov_method_sorted_list:
    print((xc_rel_ene_mad['Ag']['Total'][x] + xc_rel_ene_mad['Au']['Total'][x] + xc_rel_ene_mad['Cu']['Total'][x])/3,xc_rel_ene_mad['Total']['Total'][x])
# print([ (xc_rel_ene_mad['Ag']['Total'][x] + xc_rel_ene_mad['Au']['Total'][x] + xc_rel_ene_mad['Cu']['Total'][x])/3 for x in eov_method_sorted_list], [xc_rel_ene_mad['Total']['Total'][x] for x in eov_method_sorted_list])

199.4339831527975 199.4339831527975
152.606379898664 152.606379898664
152.606379898664 152.606379898664
142.89887646256486 142.89887646256486
129.56216720330548 129.56216720330548
189.92743278242668 189.92743278242668
190.67397646570657 190.67397646570657
120.14120310880718 120.14120310880718
123.35296088658544 123.35296088658544
111.02066718288167 111.02066718288167
359.24416494019096 359.24416494019096
138.53399311947877 138.53399311947877
103.35732006366578 103.35732006366578
126.39968425039848 126.39968425039848
119.14804054669865 119.14804054669865
121.22378536151741 121.22378536151741
137.73897047260635 137.73897047260635
190.50274081733914 190.50274081733914
117.57896711363526 117.57896711363526
122.46989674326504 122.46989674326504
182.75151155807944 182.75151155807944
341.0586959430724 341.0586959430724
341.0586959430724 341.0586959430724
224.3840900171466 224.3840900171466
85.8082174456656 85.8082174456656
146.8251578605267 146.8251578605267
277.52935434600914 277.52935434600

In [32]:
[ (xc_rel_ene_mad['Au']['Total'][x]) for x in eov_method_sorted_list] 

[265.7408397504937,
 201.34904923242968,
 201.34904923242968,
 188.77454308382616,
 181.9631253060479,
 254.19797308382545,
 253.73288752827094,
 166.23063308382538,
 191.19868086160417,
 157.75937863938205,
 352.6539697504929,
 147.5695349982402,
 128.48772166490716,
 231.15724975048533,
 217.04567086160958,
 220.90522863939336,
 279.63467800624454,
 227.0287226350016,
 144.0438859683359,
 147.02868930166878,
 214.45331041277908,
 329.02570923242894,
 329.02570923242894,
 211.68517478798526,
 105.4235433690656,
 149.84674001814082,
 277.5819256457707,
 254.15585509823686]

In [42]:
np.concatenate((np.arange(len(gga_xc_list)), 11 + np.arange(len(vdw_inc_xc_list)), 18 + np.arange(len(metagga_xc_list)), 27 + np.arange(len(hybrid_xc_list)), 39 + np.arange(len(cwft_xc_list))))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 18, 19, 20, 21,
       27, 28, 29, 30, 31, 32, 33, 34, 39, 40, 41])

In [40]:
eov_method_sorted_list_val

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])

In [63]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D



def radar_factory(num_vars, frame='circle',rotation=0):
    """
    Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle', 'polygon'}
        Shape of frame surrounding axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False) 

    class RadarTransform(PolarAxes.PolarTransform):

        def transform_path_non_affine(self, path):
            # Paths with non-unit interpolation steps correspond to gridlines,
            # in which case we force interpolation (to defeat PolarTransform's
            # autoconversion to circular arcs).
            if path._interpolation_steps > 1:
                path = path.interpolated(num_vars)
            return Path(self.transform(path.vertices), path.codes)

    class RadarAxes(PolarAxes):

        name = 'radar'
        PolarTransform = RadarTransform

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')
            self.set_theta_offset(rotation)


        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.append(x, x[0])
                y = np.append(y, y[0])
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k",orientation=rotation)
                # return RegularPolygon((0.5, 0.5), num_vars,
                #                       radius=.5, edgecolor="k")
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                # spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                #                     + self.transAxes)
                spine.set_transform(Affine2D().rotate(rotation).scale(.5).translate(.5, .5)
                                    + self.transAxes)
                # print(spine)
                return {'polar': spine}
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

dimer={0: 'OV', 1: 'MV', 2: 'HV', 3: 'HD',
       4: 'OH', 5: 'MH', 6: 'OD', 7: 'MD'
      }


def example_data():
    data = [
        [dimer[0], dimer[1], dimer[2], dimer[3], 
         dimer[4], dimer[5], dimer[6], dimer[7]],
        ('PBE+vdW empirical', [
            [m2_mgo_xc_bind_ene_mad['Total'][x]['PBE-D2'] for x in list(range(1,9))],
            [m2_mgo_xc_bind_ene_mad['Total'][x]['PBE-D2-Ne'] for x in list(range(1,9))],
            [m2_mgo_xc_bind_ene_mad['Total'][x]['PBE-D3'] for x in list(range(1,9))],
            [m2_mgo_xc_bind_ene_mad['Total'][x]['PBE-D4'] for x in list(range(1,9))],
            [m2_mgo_xc_bind_ene_mad['Total'][x]['PBE-DDSC'] for x in list(range(1,9))],
            [m2_mgo_xc_bind_ene_mad['Total'][x]['PBE-TS'] for x in list(range(1,9))],
            [m2_mgo_xc_bind_ene_mad['Total'][x]['PBE-TSHI'] for x in list(range(1,9))],
            [m2_mgo_xc_bind_ene_mad['Total'][x]['PBE-MBDHI'] for x in list(range(1,9))],
            [m2_mgo_xc_bind_ene_mad['Total'][x]['PBE-MBDFI'] for x in list(range(1,9))]
        ])
    ]
    return data



N = 8
theta = radar_factory(N, frame='polygon',rotation=5*np.pi/8)
# theta = [x+np.pi/8 for x in theta]
# print(theta)

data = example_data()
spoke_labels = data.pop(0)

fig, axs = plt.subplots(figsize=(89*mm_to_inch,70*mm_to_inch),dpi=1200,subplot_kw=dict(projection='radar'))
fig.subplots_adjust(wspace=0.25, hspace=0.20, top=0.85, bottom=0.05)

colors = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',]
axs.set_rgrids([200,400,600,800])
# axs.set_title(r'$E_\textrm{int}^\textrm{M-S}$ error w.r.t. CCSD(T) (meV)', size=11, position=(0.5, 1.1),
                # horizontalalignment='center', verticalalignment='center')

colors = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8','C9']


for index, i in enumerate(['PBE-D2', 'PBE-D2-Ne', 'PBE-D3', 'PBE-D4','PBE-DDSC', 'PBE-TS', 'PBE-TSHI', 'PBE-MBDHI','PBE-MBDFI']):
    print(index,i)
    if 'TSHI' in i:
        axs.plot(theta, [m2_mgo_xc_bind_ene_mad['Total'][x][i] for x in list(range(1,9))],'--',color=colors[index],linewidth=1)
        axs.fill(theta, [m2_mgo_xc_bind_ene_mad['Total'][x][i] for x in list(range(1,9))], facecolor=colors[index], alpha=0.25, label='_nolegend_')
    else:
        axs.plot(theta, [m2_mgo_xc_bind_ene_mad['Total'][x][i] for x in list(range(1,9))],color=colors[index],linewidth=1, alpha=0.7)
        axs.fill(theta, [m2_mgo_xc_bind_ene_mad['Total'][x][i] for x in list(range(1,9))], facecolor=colors[index], alpha=0.25, label='_nolegend_')            

axs.set_varlabels(spoke_labels)
# axs.set_theta_offset(-np.pi/8)

plt.xticks(fontsize=11)
plt.yticks(fontsize=11)


# add legend relative to top-left plot
labels = ('PBE-D2', 'PBE-D2-Ne', 'PBE-D3', 'PBE-D4','PBE-DDsC', 'PBE-TS', 'PBE-TSHI', 'PBE-MBDHI', 'PBE-MBDFI')
legend = axs.legend(labels,ncol=2, fontsize=11,loc='lower center', bbox_to_anchor=(-1, 0.3),frameon=False) # loc=(0.9, .9),
                        #   labelspacing=0.1, fontsize='large')


plt.ylim([0.0,800])


plt.savefig('M2_MgO_PBE_spider_plot.png',dpi=1200,bbox_inches = "tight")

0 PBE-D2
1 PBE-D2-Ne
2 PBE-D3
3 PBE-D4
4 PBE-DDSC
5 PBE-TS
6 PBE-TSHI
7 PBE-MBDHI
8 PBE-MBDFI


In [ ]:
[m2_mgo_xc_bind_ene_mad['Total'][x][i] for x in list(range(1,9))]

[63.38997635337913,
 166.61582877342582,
 133.75127282063906,
 179.9903120797879,
 191.0152745174563,
 200.18028961996228,
 220.1530895929609,
 182.02386198838497]

In [ ]:
[m2_mgo_xc_bind_ene_mad['Cu'][x] for x in xc_functional_list + ['HF','MP2','CCSD']]

[49.95237776070641,
 348.3262403846173,
 348.3262403846173,
 287.3608616345532,
 401.9015091345616,
 544.3520341344888,
 197.40522663444483,
 274.1348866346068,
 211.08621788458535,
 413.68319038459083,
 311.79346836556573,
 549.660705384573,
 250.42258288447718,
 188.32554788451725,
 615.7954653844881,
 392.6735053846228,
 94.65071088065703,
 99.90942588068418,
 244.5518478845173,
 135.54717663450253,
 185.11775288451202,
 413.6428866344284,
 320.36592788456767,
 364.27806788457093,
 210.19918961536166,
 278.5381991345032,
 102.4663978845545,
 41.04686910531616,
 215.28729836544187,
 92.94101971096836,
 142.17411977893596,
 142.17411977893596,
 410.450690384473,
 453.25902663446584,
 937.1746726868378,
 42.14324077261681,
 254.44042154943378]